In [1]:
%matplotlib inline
import os
import re
import sys
from oqmbt.oqt_project import OQtProject
from oqmbt.notebooks.sources_shallow_fault.create_fault_sources_from_geojson import get_fault_sources
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.geo.surface import SimpleFaultSurface
from openquake.hazardlib.mfd import EvenlyDiscretizedMFD
from openquake.hazardlib.scalerel.wc1994 import WC1994
from openquake.hazardlib.sourcewriter import write_source_model

In [2]:
prj_path = "/Users/kjohnson/GEM/Regions/paisl18/project_8/paisl.oqmbtp"
os.environ["OQMBT_PROJECT"] = prj_path

In [3]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]

## Parameters

In [4]:
slip_rate_class = 'suggested' 
m_low = model.faults_lower_threshold_magnitude
bin_width = model.mfd_binwidth
scalerel_name = model.msr
rupture_mesh_spacing = model.fault_rupture_mesh_spacing
upper_seismogenic_depth = model.upper_seismogenic_depth
## THIS PARAMETER IS NOT USED BUT THIS PART NEEDS TO BE IMPROVED
lower_seismogenic_depth=10.0 
rupture_aspect_ratio=2.0
temporal_occurrence_model=PoissonTOM(1.0)
aseismic_coeff=0.9
# this is the geojson file
#filename = os.path.abspath(os.path.join(oqtkp.directory, model.fault_geojson_filename))
filename = '/Users/kjohnson/GEM/Regions/paisl18/data/gem-global-active-faults/ModelFaults.geojson'
#filename = '/Users/kjohnson/GEM/Regions/paisl18/data/gem-global-active-faults/cf3_testing.geojson'

## Set the magnitude scaling relationship
The options available for the magnitude scaling relationship are:
* `scalrel_name = 'WC1994`' <br> Wells and Coppersmith (1994) scaling relationship
* '`PointMSR`' Scaling relationship to be used for the creation of tiny ruptures resembling points
* '`CEUS2011`' Scaling relationship developed within the CEUS SSC project

In [5]:
import importlib
module = importlib.import_module('openquake.hazardlib.scalerel')
my_class = getattr(module, scalerel_name)
msr = my_class()

In [6]:
from openquake.hazardlib.scalerel.wc1994 import WC1994
from openquake.hazardlib.tom import PoissonTOM
m_low=6.5 
b_gr=1.0
rupture_mesh_spacing=2.0 
upper_seismogenic_depth=0.0
lower_seismogenic_depth=10.0 
msr=WC1994()
rupture_aspect_ratio=2. 
temporal_occurrence_model=PoissonTOM(1.0)
aseismic_coeff=0.9
oqsource=False

In [7]:
import logging
import json
logging.info('Reading %s and slip_type = %s' % (filename, slip_rate_class))
with open(filename, 'r') as data_file:
    data = json.load(data_file)

In [8]:
def get_tples(tstr):
    """
    Extract information included in the tuples contained in the .geojson file

    :parameter string tstr:
        The string with the tuple
    :returns:
        A list containing the values of the tuple
    """
    if tstr is not None and len(tstr):
        tstra = re.sub('\(', '', re.sub('\)', '', tstr))
        flist = []
        for tmp in re.split('\,', tstra):
            if re.search('[0-9]', tmp):
                flist.append(float(tmp))
            else:
                flist.append(None)
    else:
        flist = (None, None, None)
    flista = tuple(flist)
    return flista


In [9]:
from openquake.hazardlib.geo.point import Point
def get_line(dat):
    plist = []
    for tple in dat:
        xa = tple[0]
        #print(tple)
        #print(xa)
        #print("gap")
        x = xa-360 if xa>180 else xa
        #print(type(tple))
        #print(Point(x, tple[1]))
        plist.append(Point(x, tple[1]))
    return plist

In [10]:
from openquake.hazardlib.const import TRT
import numpy
srcl = []
for idf, feature in enumerate(data['features']):

    source_id = '{0:d}'.format(idf)
    tectonic_region_type = TRT.ACTIVE_SHALLOW_CRUST

    fs_name = ''
    ns_name = ''

    # get fault name[s] - id
    if feature['properties']['fs_name'] is not None:
        fs_name = feature['properties']['fs_name']
    if feature['properties']['fs_name'] is not None:
        ns_name = feature['properties']['fs_name']
    name = '{0:s} | {1:s}'.format(fs_name, ns_name)
    if 'ogc_fid' in feature['properties']:
        id_fault = feature['properties']['ogc_fid']
    else:
        id_fault = '%d' % (idf)
    tmplist = (numpy.array(feature['geometry']['coordinates'])) 
    if len(tmplist[0])>2:
        fault_trace = get_line(tmplist[0])
    else:
        fault_trace = get_line(tmplist)
    #print(tmplist[0])
# Get fault trace geometry
    #fault_trace = get_line(numpy.array(feature['geometry']['coordinates']))
#    fault_trace = get_line(tmplist[0])

In [11]:
print(fault_trace)

[<Latitude=-3.656000, Longitude=151.301000, Depth=0.0000>, <Latitude=-3.661000, Longitude=151.308400, Depth=0.0000>, <Latitude=-3.666000, Longitude=151.315799, Depth=0.0000>, <Latitude=-3.671001, Longitude=151.323199, Depth=0.0000>, <Latitude=-3.676001, Longitude=151.330599, Depth=0.0000>, <Latitude=-3.681001, Longitude=151.337999, Depth=0.0000>, <Latitude=-3.686001, Longitude=151.345399, Depth=0.0000>, <Latitude=-3.691001, Longitude=151.352799, Depth=0.0000>, <Latitude=-3.696000, Longitude=151.360199, Depth=0.0000>, <Latitude=-3.701000, Longitude=151.367600, Depth=0.0000>, <Latitude=-3.706000, Longitude=151.375000, Depth=0.0000>, <Latitude=-3.712000, Longitude=151.381625, Depth=0.0000>, <Latitude=-3.718000, Longitude=151.388249, Depth=0.0000>, <Latitude=-3.724000, Longitude=151.394874, Depth=0.0000>, <Latitude=-3.730000, Longitude=151.401499, Depth=0.0000>, <Latitude=-3.736000, Longitude=151.408124, Depth=0.0000>, <Latitude=-3.742000, Longitude=151.414749, Depth=0.0000>, <Latitude=-3.

## Creating the xml file

In [12]:
b_gr = 1.0 # this is a dummy value that will be modified during following steps
sources = get_fault_sources(filename, 
                            slip_rate_class, 
                            bin_width, 
                            m_low, 
                            b_gr,
                            rupture_mesh_spacing, 
                            upper_seismogenic_depth,
                            lower_seismogenic_depth, 
                            msr,
                            rupture_aspect_ratio, 
                            temporal_occurrence_model,
                            aseismic_coeff)

------------------------------------------------------------------------------
Sinistral
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [140.4, None, None]
shortening_rate=  [11.8, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [139.9, None, None]
mech: Sinistral
Dip value for id= 0 is missing and was computed using slipt= Sinistral, new dip= 90.0 
Rake value for id= 0 is missing and was computed using slipt= Sinistral, new rake= 0 
net_slip value for id= 0 is net_slip= 126.36000000000001 [slipr = Sinistral] 
mean_azimuth=  149.537709722
rld= 180.355107469, width = 17.0, class= Sinistral
slip_rate=  126.36000000000001
right
Reverse
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [10.05, None, None]
shortening_rate=  [5.55, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [8.4, None, None]
mech: Reverse
Dip value for id= 1 is missing and was computed using slipt= Reverse, n

 [13.0, None, None]
shortening_rate=  [0.0, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [13.0, None, None]
mech: Sinistral
Dip value for id= 2 is missing and was computed using slipt= Sinistral, new dip= 90.0 
Rake value for id= 2 is missing and was computed using slipt= Sinistral, new rake= 0 
net_slip value for id= 2 is net_slip= 11.700000000000001 [slipr = Sinistral] 
mean_azimuth=  229.794589489
rld= 216.839056272, width = 17.0, class= Sinistral
slip_rate=  11.700000000000001
right
Dextral
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [40.2, None, None]
shortening_rate=  [-3.1, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [40.1, None, None]
mech: Dextral
Dip value for id= 3 is missing and was computed using slipt= Dextral, new dip= 90.0 
Rake value for id= 3 is missing and was computed using slipt= Dextral, new rake= 180 
net_slip value for id= 3 is net_slip= 36.18000000000001 [slipr = Dextral]


dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [38.8, None, None]
shortening_rate=  [-37.6, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [9.5, None, None]
mech: Spreading
Dip value for id= 8 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 8 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 8 is net_slip= 5.819999999999999 [slipr = Spreading Ridge] 
mean_azimuth=  67.2466904757
slip_rate=  5.819999999999999
Spreading Ridge
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [38.5, None, None]
shortening_rate=  [-38.4, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [2.6, None, None]
mech: Spreading
Dip value for id= 9 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 9 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 

raket=  (None, None, None)
sliprt=  [36.7, None, None]
shortening_rate=  [-35.9, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [7.6, None, None]
mech: Spreading
Dip value for id= 16 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 16 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 16 is net_slip= 5.505 [slipr = Spreading Ridge] 
mean_azimuth=  70.1628148586
slip_rate=  5.505
Sinistral/Transform
found one
Sinistral/Transform
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [37.0, None, None]
shortening_rate=  [-9.9, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [35.6, None, None]
mech: Sinistral
Dip value for id= 17 is missing and was computed using slipt= Sinistral/Transform, new dip= 90.0 
Rake value for id= 17 is missing and was computed using slipt= Sinistral/Transform, new rake= 180 
net_slip value for id= 17 i


dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [30.2, None, None]
shortening_rate=  [-29.3, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [4.8, None, None]
mech: Spreading
Dip value for id= 22 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 22 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 22 is net_slip= 4.529999999999999 [slipr = Spreading Ridge] 
mean_azimuth=  85.5261736324
slip_rate=  4.529999999999999
right
Spreading Ridge
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [26.1, None, None]
shortening_rate=  [-25.5, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [5.4, None, None]
mech: Spreading
Dip value for id= 23 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 23 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip val

...

N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [22.2, None, None]
shortening_rate=  [-10.9, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [19.3, None, None]
mech: Spreading
Dip value for id= 31 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 31 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 31 is net_slip= 3.3299999999999996 [slipr = Spreading Ridge] 
mean_azimuth=  102.885957235
slip_rate=  3.3299999999999996
right
Spreading Ridge
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [22.2, None, None]
shortening_rate=  [-9.6, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [20.1, None, None]
mech: Spreading
Dip value for id= 32 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 32 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip

(None, None, None)
raket=  (None, None, None)
sliprt=  [21.2, None, None]
shortening_rate=  [-17.5, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [10.8, None, None]
mech: Spreading
Dip value for id= 36 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 36 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 36 is net_slip= 3.1799999999999997 [slipr = Spreading Ridge] 
mean_azimuth=  259.915892416
slip_rate=  3.1799999999999997
right
Dextral/Transform
found one
Dextral/Transform
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [17.5, None, None]
shortening_rate=  [4.7, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [16.8, None, None]
mech: Dextral
Dip value for id= 37 is missing and was computed using slipt= Dextral/Transform, new dip= 90.0 
Rake value for id= 37 is missing and was computed using slipt= Dextral/Transform, n

dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [5.7, None, None]
shortening_rate=  [-4.9, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [2.9, None, None]
mech: Spreading
Dip value for id= 42 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 42 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 42 is net_slip= 0.855 [slipr = Spreading Ridge] 
mean_azimuth=  274.387402468
slip_rate=  0.855
right
Spreading Ridge
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [4.2, None, None]
shortening_rate=  [-2.2, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [3.5, None, None]
mech: Spreading
Dip value for id= 43 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 43 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id

 [3.7, None, None]
mech: Spreading
Dip value for id= 46 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 46 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 46 is net_slip= 4.29 [slipr = Spreading Ridge] 
mean_azimuth=  157.37448106
slip_rate=  4.29
right
Spreading Ridge
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
sliprt=  [34.2, None, None]
shortening_rate=  [-34.0, None, None]
vertical_slip_rate=  (None, None, None)
strike_slip_rate=  [2.9, None, None]
mech: Spreading
Dip value for id= 47 is missing and was computed using slipt= Spreading Ridge, new dip= 60.0 
Rake value for id= 47 is missing and was computed using slipt= Spreading Ridge, new rake= -90 
net_slip value for id= 47 is net_slip= 5.13 [slipr = Spreading Ridge] 
mean_azimuth=  156.652020772
slip_rate=  5.13
right
Spreading Ridge
dip_dir fazzula=  N
dipt=  (None, None, None)
raket=  (None, None, None)
slipr

ValueError: unsupported style of slip fault Sinistral/Transofrm

In [ ]:
print(sources[0])

In [ ]:
for src in sources:
    key = 'sf_{:s}'.format(src.source_id)
    assert key not in model.sources
    src.source_id = key
    model.sources[key] = src
oqtkp.models[model_id] = model
oqtkp.save()